In [4]:
import numpy as np 
import torch 
import pandas as pd
import yfinance as yf
from arch import arch_model
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from scipy.optimize import minimize
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
import pytorch_lightning as pl


In [5]:
def generate_ground_garch(omega, alpha, beta, n=1000):
    
    am = arch_model(None, mean='Zero', vol='GARCH', p=1, q=1, power = 2) #Остатки просто получаются умножением волатильности на кси ~N(0,1)
    params = np.array([omega, alpha, beta])
    am_data = am.simulate(params, n)

    return am_data['data'].to_numpy()*100, am_data['volatility'].to_numpy()*100

In [6]:
omega, alpha, beta = 0.2, 0.2, 0.3

In [7]:
class CustomSyntDataset(Dataset):

    def __init__(self, omega, alpha, beta, n =1000):
        self.omega = omega
        self.alpha = alpha
        self.beta = beta
        self.n = n
        self.residuals, self.volatility = generate_ground_garch(omega, alpha, beta, n)

        self.inputs = np.column_stack([
            np.full_like(self.residuals, 1),
            np.square(self.residuals),
            np.square(self.volatility)
        ])
        
        self.outputs = np.square(np.roll(self.volatility, -1))

        self.inputs = torch.Tensor(self.inputs)

        self.outputs = torch.Tensor(self.outputs)

    def __len__(self):
        return self.n -1 
    
    def __getitem__(self, index):
        return self.inputs[index], self.outputs[index], self.inputs[index+1, 1]

In [8]:
ds = CustomSyntDataset(omega=omega, alpha=alpha, beta = beta)

In [9]:
for i in range(1, 5):
    print(ds[i])

(tensor([1.0000e+00, 8.0578e+02, 3.0214e+03]), tensor(3067.5625), tensor(1486.2452))
(tensor([1.0000e+00, 1.4862e+03, 3.0676e+03]), tensor(3217.5178), tensor(14.2149))
(tensor([1.0000e+00, 1.4215e+01, 3.2175e+03]), tensor(2968.0984), tensor(1760.9266))
(tensor([1.0000e+00, 1.7609e+03, 2.9681e+03]), tensor(3242.6147), tensor(911.6739))


In [10]:
dl = DataLoader(ds, batch_size = 64, shuffle= False, drop_last=True)

In [11]:
class GarchNN(torch.nn.Module):
    def __init__(self):
        super(GarchNN, self).__init__()
        self.model =nn.Sequential(
            nn.Flatten(),
            nn.Linear(3,1, bias=False)
        )

    def forward (self, x):
        return self.model(x).squeeze(-1)

In [12]:
class NLoss(torch.nn.Module):
    def __init__(self):
        super(NLoss, self).__init__()

    def forward(self, pred_volatility, target_resid):
        left_side = torch.log(pred_volatility)/2
        right_side = (target_resid/pred_volatility) /2

        return (left_side + right_side).sum()
                

In [13]:
model = GarchNN()
optimizer = torch.optim.Adam(model.parameters(), lr =1e-3)
criterion = NLoss()
#criterion = nn.MSELoss()

In [14]:
num_epochs = 1000

for epochs in tqdm(range(num_epochs), desc="Training"):
    epoch_loss =0.0
    model.train()

    for inputs, targets, resids in dl:
        optimizer.zero_grad()
        output = model(inputs)
        loss = criterion(output, resids)
        loss.backward()
        optimizer.step()

        epoch_loss+=loss.item()

    avg_loss = epoch_loss/len(dl)

    tqdm.write(f"Epoch {epochs+1}/{num_epochs} | Loss: {avg_loss:.4f}")
    

Training:   0%|          | 0/1000 [00:00<?, ?it/s]

Epoch 1/1000 | Loss: nan
Epoch 2/1000 | Loss: nan
Epoch 3/1000 | Loss: nan
Epoch 4/1000 | Loss: nan
Epoch 5/1000 | Loss: nan
Epoch 6/1000 | Loss: nan
Epoch 7/1000 | Loss: nan
Epoch 8/1000 | Loss: nan
Epoch 9/1000 | Loss: nan
Epoch 10/1000 | Loss: nan
Epoch 11/1000 | Loss: nan
Epoch 12/1000 | Loss: nan
Epoch 13/1000 | Loss: nan
Epoch 14/1000 | Loss: nan
Epoch 15/1000 | Loss: nan
Epoch 16/1000 | Loss: nan
Epoch 17/1000 | Loss: nan
Epoch 18/1000 | Loss: nan
Epoch 19/1000 | Loss: nan
Epoch 20/1000 | Loss: nan
Epoch 21/1000 | Loss: nan
Epoch 22/1000 | Loss: nan
Epoch 23/1000 | Loss: nan
Epoch 24/1000 | Loss: nan
Epoch 25/1000 | Loss: nan
Epoch 26/1000 | Loss: nan
Epoch 27/1000 | Loss: nan
Epoch 28/1000 | Loss: nan
Epoch 29/1000 | Loss: nan
Epoch 30/1000 | Loss: nan
Epoch 31/1000 | Loss: nan
Epoch 32/1000 | Loss: nan
Epoch 33/1000 | Loss: nan
Epoch 34/1000 | Loss: nan
Epoch 35/1000 | Loss: nan
Epoch 36/1000 | Loss: nan
Epoch 37/1000 | Loss: nan
Epoch 38/1000 | Loss: nan
Epoch 39/1000 | Loss:

In [15]:
print(torch.isnan(ds.inputs).any(), torch.isinf(ds.inputs).any())
print(torch.isnan(ds.outputs).any(), torch.isinf(ds.outputs).any())

tensor(False) tensor(False)
tensor(False) tensor(False)


In [16]:
def get_model_weights(model):
    weights = {}
    for name, param in model.named_parameters():
        weights[name] = param.data.clone().cpu().numpy()
    return weights

model_weights = get_model_weights(model)

In [17]:
model_weights

{'model.1.weight': array([[0.38470533, 0.19164675, 0.0050107 ]], dtype=float32)}